In [1]:
# using Pkg
# Pkg.activate("/home/jek354/research/ML-signproblem")
# Pkg.update()

using Lattices
using LinearAlgebra
using Combinatorics
using SparseArrays
using Plots
import Graphs
using LaTeXStrings
import Statistics
using Random
using Zygote
using Optimization, OptimizationOptimisers
using JSON
using OptimizationOptimJL
using JLD2
# using CombDiff
# using ExponentialUtilities


include("ed_objects.jl")
include("ed_functions.jl")
include("ed_optimization.jl")
include("utility_functions.jl")
include("adiabatic_analysis.jl")

:((M, v_1, v_2, matexp)->begin
          #= /Users/jonathonkambulow/.julia/packages/CombDiff/ta3jt/src/extension/codegen.jl:109 =#
          #= /Users/jonathonkambulow/.julia/packages/CombDiff/ta3jt/src/extension/codegen.jl:111 =#
          (A, _k)->begin
                  #= /Users/jonathonkambulow/.julia/packages/CombDiff/ta3jt/src/extension/codegen.jl:109 =#
                  #= /Users/jonathonkambulow/.julia/packages/CombDiff/ta3jt/src/extension/codegen.jl:111 =#
                  (_i,)->begin
                          #= /Users/jonathonkambulow/.julia/packages/CombDiff/ta3jt/src/extension/codegen.jl:109 =#
                          #= /Users/jonathonkambulow/.julia/packages/CombDiff/ta3jt/src/extension/codegen.jl:111 =#
                          0
                      end
              end
      end)

time_ordered_unitary

In [1]:
using CombDiff
using LinearAlgebra
using InteractiveUtils
# Monkey-patch CombDiff.codegen to prevent generation of `_` variable which causes syntax errors.
function CombDiff.codegen(v::CombDiff.Var)
    s = CombDiff.name(v)
    if s == :_
        return :_safe_var_replaced
    end
    return s
end
# Workaround for renamed variable
struct CombDiffIgnore end
const combdiff_ignore = CombDiffIgnore()
const _safe_var_replaced = combdiff_ignore
println("Starting CombDiff Minimal Working Example (Updated)")
try
    println("Compiling dot product gradient...")
    
    # 1. Define the spaces and the function in one @pct block
    f, ctx = CombDiff.@pct begin
        @space V begin
            type = (N,) -> C
        end
        # Function: f(v) = pullback( (x) -> sum(v .* x) )
        (v::V) -> pullback((x::V) -> sum(i::N, v(i) * x(i)))
    end
    
    # 2. Evaluate the pullback and compile
    df = CombDiff.eval_pullback(f)
    generated_code = CombDiff.codegen(df)
    dot_grad_gen = invokelatest(eval(generated_code), nothing)
    println("Compilation successful!")
    # 3. Test values
    v_val = ComplexF64[1.0, 2.0, 3.0]
    grad_func = invokelatest(dot_grad_gen, v_val)
    
    x_val = ComplexF64[0.1, 0.2, 0.3] # Value not actually used for linear function pullback
    k_seed = 1.0 + 0.0im
    
    # grad_func(x, seed) -> result_map (a function of index)
    res_map = invokelatest(grad_func, x_val, k_seed)
    println("Result map type: ", typeof(res_map))
    
    # res_map(i) should be v(i) * seed
    for i in 1:3
        val = invokelatest(res_map, i)
        println("Grad at index $i: ", val, " (Expected: ", v_val[i], ")")
    end
catch e
    println("\n!!! Error occurred !!!")
    showerror(stdout, e)
    println()
    Base.show_backtrace(stdout, catch_backtrace())
    println()
end
println("\n--- SCALING UP TO MATRIX EXPONENTIAL ---")
try
    # Defining the actual target function structure
    f_complex, ctx_complex = CombDiff.@pct begin
        @space RV begin type = (N,) -> C end
        @space R3 begin type = (N,N,N) -> C end
        @space RM begin type = (N,N) -> C end
        @space Matfun begin type = (RM,) -> RM end
        (M::R3, v_1::RV, v_2::RV, matexp::Matfun) -> pullback((A::RV) ->
            sum((i::N, j::N), v_1(i) * matexp(
                                  (p::N, q::N) -> sum(k::N, A(k) * M(k, p, q))
                              )(i, j) * v_2(j))
        )
    end
    println("Matrix exponential function f defined.")
    df_complex = CombDiff.eval_pullback(f_complex)
    println("Pullback evaluated.")
    
    gen_code_complex = CombDiff.codegen(df_complex)
    println("Codegen successful.")
    # println(gen_code_complex)
    grad_gen_complex = invokelatest(eval(gen_code_complex), nothing)
    println("Matrix exponential gradient compiled!")
catch e
    println("\n!!! Matrix Complex Error occurred !!!")
    showerror(stdout, e)
    println()
end


Starting CombDiff Minimal Working Example (Updated)
Compiling dot product gradient...
Compilation successful!

!!! Error occurred !!!
MethodError: no method matching (::var"#27#28")(::Vector{ComplexF64})
The function `#27` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  (::var"#27#28")(::Any, ::Any)
   @ Main ~/.julia/packages/CombDiff/ta3jt/src/extension/codegen.jl:109


Stacktrace:
  [1] top-level scope
    @ ~/Library/CloudStorage/Dropbox/programming/cornell courses/research/experimenting/ed/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X12sZmlsZQ==.jl:36
  [2] eval(m::Module, e::Any)
    @ Core ./boot.jl:489
  [3] include_string(mapexpr::typeof(REPL.softscope), mod::Module, code::String, filename::String)
    @ Base ./loading.jl:2870
  [4] (::VSCodeServer.var"#notebook_runcell_request##0#notebook_runcell_request##1"{VSCodeServer.NotebookRunCellArguments, String})()
    @ VSCodeServer ~/.antigravity/extensions/julialang.languag

In [10]:
M = rand(2,2,2)
x = [1.0, 0]
y = [0.0, 1]
my_matexp(x) = exp(1im * x)
grad_func = COMBDIF_GRAD_GEN(M, x, y, my_matexp)
grad_func([1.0 1.4], 1.0 + 0im)(0)

0

In [2]:
# using KrylovKit


t = 1.0
U = 6
μ = 0  # positive incentivises fewer particles (one electron costs this much energy)
N_up = 4
N_down = 4
N =  3
half_filling = false
# lattice = Chain(6, Periodic())
lattice_dimension = (2,3)
bc = "periodic"
# lattice = Chain(6, Periodic())
lattice = Square(lattice_dimension, if bc == "periodic" Periodic() else Open() end)
# lattice = Graphs.cycle_graph(3)

models = []

reference_index = 2
# t_values = []# Float64[1,1], Float64[1,0.1], 1.0]
# for _t in t_values
#     # println(_t)
#     push!(models,HubbardModel(_t,0.0001,μ,half_filling))
# end
# U_values = [0.00001,0.01]
U_values = [0.00001; LinRange(2.1,9,20)]
U_values = sort([U_values; 10.0 .^LinRange(-3,2,40)])

for U in U_values
    # println(t)
    push!(models,HubbardModel(t,U,μ,half_filling))
end

subspace = HubbardSubspace(N, lattice)
# subspace = HubbardSubspace(N_up, N_down, lattice)

ops = []
if subspace.N >= 1
    push!(ops,create_operator(subspace,:Sx))
end
push!(ops, create_operator(subspace,:S2))
# op3 = Matrix(create_operator(subspace,:L2))
push!(ops, create_operator(subspace,:T, kind=1))
push!(ops, create_operator(subspace,:T, kind=2))

H = []
for model ∈ models
    push!(H, Matrix(create_Hubbard(model, subspace)))
end


# _, indexer = create_Hubbard(models[1], subspace; get_indexer=true)
# degen_rm_U_1 = create_consistent_basis(H, ops;reference_index=reference_index)

In [7]:
include("ed_objects.jl")
include("ed_functions.jl")
include("ed_optimization.jl")
include("utility_functions.jl")

dic = load_saved_dict("data/N=3_2x3/meta_data_and_E.jld2")

meta_data = dic["meta_data"]
U_values = meta_data["U_values"]
all_full_eig_vecs = dic["all_full_eig_vecs"]
n_eigs = [parse(Int,x) for x in split(meta_data["sites"], "x")]
indexer = dic["indexer"]

# meta_data = Dict("electron count"=>3, "sites"=>"2x3", "bc"=>"periodic", "basis"=>"adiabatic", 
#                 "U_values"=>U_values, "maxiters"=>10)
instructions = Dict("starting state"=>Dict("U index"=>1, "levels"=>1),
                "ending state"=>Dict("U index"=>30, "levels"=>1), "max_order"=>2, "use symmetry"=>false)
println("U in [",U_values[instructions["starting state"]["U index"]], ", ", U_values[instructions["ending state"]["U index"]],"]")

# x = zero(degen_rm_U[1:2,:])
# x[1,1] = 1
# x[2,2] = 1
data_dict_tmp = test_map_to_state(all_full_eig_vecs[6], instructions, indexer, !isa(meta_data["electron count"], Number);
     maxiters=30#meta_data["maxiters"]
     , optimization=:combdiff)
data_dict_tmp
# save_with_metadata(data_dict_tmp, "data/tmp.jld2")

U in [1.0e-5, 2.463157894736842]
magnitude: 0.013941348582661217
learning rate: 0.0027882697165322436
  0.160246 seconds (334.91 k allocations: 15.361 MiB, 99.94% compilation time)
  0.082022 seconds (365.43 k allocations: 34.987 MiB, 9.83% gc time, 83.30% compilation time)


UndefVarError: UndefVarError: `local_ctx` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [107]:
tmp1 = data_dict_tmp["coefficients"]

1-element Vector{Any}:
 Any[[0.00804608274539668, -0.0035627311308626727, 0.013194835069913913, -0.019268140850836987, -0.0014423204630328395, -0.0014497388671231462, -0.000581176258498751, 0.004805596792375161, -0.0006780557731068106, -0.00483712196787348  …  0.0028166715512127115, 0.013390362781723337, 0.0053384682331121735, 0.0010069621609959313, 0.0019766507430968986, -0.005283564663695769, -0.009784655021768559, -0.0010179350165393398, 0.013508137220184915, -0.002029387141122564]]

In [109]:
tmp2 =  data_dict_tmp["coefficients"]

1-element Vector{Any}:
 Any[[0.004102925388190125, 0.006857089225758785, 0.010862586647674527, -0.015113738591211994, 0.0020029422241328717, -0.003917419109382482, 0.0006817613154320765, -0.0027356295135367104, 0.003944283685439162, -0.0011206962068358818  …  -0.00445876724572687, 0.014266343195250178, -0.0008161245510463928, 0.004458669581031902, -0.005829410058194202, -0.005148793353387051, -0.0135012837269036, 0.0014134604142347466, 0.01564486999853579, 0.0001853136543891242]]

In [91]:
h_idx = 1
sort_i = sortperm(real.(diag(degen_rm_U_1[h_idx]'*H[h_idx]*degen_rm_U_1[h_idx])))
# degen_rm_U_1[1][:,sort_i]'*H[1]*degen_rm_U_1[1][:,sort_i]
for i = 1:10
    for k = 1:6
    state1 = degen_rm_U_1[h_idx][:,sort_i[i]]
    state2 = all_full_eig_vecs[k][h_idx,:]
    if abs(state1'*state2) > 0.999 #&& abs(state1'*state2) < 0.9999
        println("$(sort_i[i]) $k")
        println("overlap: ", abs(state1'*state2))
        println("Operator expectation")
        for op in ops
            println(state1'*op*state1)
            println(state2'*op*state2)
            println()
        end
        println("Energy")
        println(state1'*H[h_idx]*state1)
        println(state2'*H[h_idx]*state2)
        println("--------")
    end
    end
end

209 6
overlap: 1.0000000000000004
Operator expectation
-0.5000000000000004 + 3.6485613330733966e-18im
-0.49999999999999994 + 2.125326019440164e-18im

0.7500000000000004 + 5.813531224141085e-18im
0.75 - 1.0918852741690807e-17im

-1.0000000000000009 - 9.697048247126494e-18im
-1.0 + 1.5846320501374278e-17im

-0.5000000000000004 + 0.8660254037844392im
-0.5 + 0.8660254037844388im

Energy
-8.999996666669809 + 1.1245712333560524e-17im
-8.999996666669803 + 1.062207109125475e-17im
--------
106 4
overlap: 1.0
Operator expectation
-0.4999999999999997 + 3.3097316105594254e-18im
-0.5000000000000001 - 8.553055805603911e-19im

0.7499999999999997 - 5.71922932558759e-18im
0.7500000000000002 - 2.646269893565343e-18im

-0.9999999999999997 + 5.152124223788585e-18im
-1.0000000000000002 + 9.103246718237953e-18im

-0.4999999999999997 - 0.8660254037844382im
-0.5000000000000001 - 0.8660254037844388im

Energy
-8.999996666669798 - 5.494215684893975e-17im
-8.999996666669803 + 8.471574875673278e-17im
--------
102 